In [5]:
import os
import json
import pandas as pd

from openai import OpenAI

## 데이터 전처리

In [2]:
# 엑셀 파일 경로
file_path = '../data/arts.xlsx'
df = pd.read_excel(file_path)
df = df.drop(columns=['_x0008_작품이미지', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'])
df["이미지"] = df['번호'].apply(lambda x: str(x)+".jpg")
df = df[0:50]
df.to_csv("../data/arts.csv", index=False)

In [3]:
df = pd.read_csv("../data/arts.csv")

### Solar API
- 작가의 설명과 작품의 설명을 분리합니다.

In [104]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리하고 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""

for i in range(len(df)):
    row_data = df.loc[i]
    content = df['작품 설명'][i]

    client = OpenAI(
        api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
    )
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content},
        ],
    )

    output_dict = json.loads(chat_result.choices[0].message.content)
    df.loc[i, 'art_description'] = output_dict['art_description']
    df.loc[i, 'author_description'] = output_dict['author_description']
    

In [115]:
# 이상치 처리
df.loc[0, 'art_description'] = "〈포도 따는 남자〉는 포도 아래 쭈그린 자세로 앉아있는 인물을 그린 작품으로, 흐릿하게 보이는 먼 공간을 표현했습니다. 포도알 하나가 인물의 손보다 크게 그려져 있어 상상적 풍경에 가까운 모습입니다. 그림 속 포도를 따는 모습은 노동의 현장이기보다는 풍요로운 동산에서 자유롭게 수확하고 있는 느낌의 그림으로 프랑스의 화가 폴 고갱(Paul Gauguin)의 타히티 시절 그림을 연상시키기도 합니다."
df.loc[0, 'author_description'] = "이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했습니다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있습니다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸습니다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌습니다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예입니다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타납니다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌습니다."

In [117]:
df.to_csv("../data/arts02.csv", index=False)

In [2]:
df = pd.read_csv("arts02.csv")

In [8]:
df

,번호,작가,작품명,제작연도,재료,규격,부문,관리번호,작품 설명,이미지,art_description,author_description
0,1,이중섭,포도 따는 남자,194,"종이에 펜, 수채 물감",14 x 9,드로잉,10090.0,"대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에...",1.jpg,"〈포도 따는 남자〉는 포도 아래 쭈그린 자세로 앉아있는 인물을 그린 작품으로, 흐릿...","이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것..."
1,2,김기창,진돗개,1975,"종이에 먹, 색",66.3 x 95.6,회화1,9172.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",2.jpg,진돗개는 조화로운 색채와 적절한 먹의 농담 표현으로 개의 모습을 생동감 있게 전달한...,운보 김기창은 어렸을 때 장티푸스에 걸려 청각장애를 갖게 되었습니다. 17세에 이당...
2,3,김기창,청산정경,1979,"비단에 먹, 색",68.2 x 136.6,회화1,9177.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",3.jpg,"청산정경은 김기창의 청록산수화로, 군청색과 녹청색 계열로 채색한 큰 산과 푸른 산하...","김기창은 어렸을 때 장티푸스에 걸려 청각장애를 갖게 되었으며, 17세에 이당 김은호..."
3,4,이응노,구상,1974,"캔버스에 천, 콜라주",204 x 126,회화1,9909.0,"고암(顧庵) 이응노(李應魯, 1904-1989)는 1922년에 동양화가인 김규진에게...",4.jpg,"이응노의 1970년대 문자추상 경향을 보여주는 대표적인 작품 중 하나로, 거친 천 ...","고암(顧庵) 이응노(李應魯, 1904-1989)는 1922년에 동양화가인 김규진에게..."
4,5,이응노,사자,1976,"종이에 먹, 색",42.7 x 59.5,회화1,9927.0,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會...",5.jpg,옅은 미색으로 칠해진 배경에 화면 왼편을 향해 선 사자의 모습이 파묵과 발묵으로 개...,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會..."
5,6,이응노,폐허의 서울,1954,"종이에 먹, 색",50 x 63.5,회화1,9958.0,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會...",6.jpg,"이응노의 작품 <폐허의 서울>은 1950년대에 그려진 작품으로, 전쟁으로 인해 폐허...",이응노는 1904년에 태어나 1989년에 사망한 한국의 화가입니다. 그는 17세 무...
6,7,이응노,달과 염소,1955,"종이에 먹, 색",26.4 x 39,회화1,9954.0,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會...",7.jpg,고암 이응노의 <달과 염소>는 푸른 달을 배경으로 서 있는 두 마리의 양을 그린 작...,고암 이응노는 1904년에 태어나 1989년에 사망한 한국의 화가입니다. 그는 17...
7,8,이용우,삼고초려,1940,"종이에 먹, 색",130.5 x 119.5,회화,9903.0,"묵로(墨鷺) 이용우(李用雨, 1902-1952)는 1911년 만 아홉 살이라는 나이...",8.jpg,"묵로(墨鷺) 이용우(李用雨, 1902-1952)는 1911년 만 아홉 살이라는 나이...",<삼고초려>는 유비가 제갈량을 초빙하기 위해 초옥을 세 번 찾아간다는 내용의 ‘삼고...
8,9,김상우,마릴린,2009,캔버스에 유화 물감,105 x 105,회화2,9199.0,"김상우(金相佑, 1971- )는 중앙대학교 미술대학 및 동 대학원을 졸업하고, 러시...",9.jpg,마릴린 먼로의 초상이 극사실주의적으로 묘사되어있는 작업입니다. 작가는 마릴린 먼로의...,"김상우(金相佑, 1971- )는 중앙대학교 미술대학 및 동 대학원을 졸업하고, 러시..."
9,10,파블로 피카소,얼굴 197번,1963,"백토, 화장토, 에나멜 장식, 유약 시유",25.5 x 25.5 x 2.8,공예,10366.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",10.jpg,"흰색 원형 접시를 배경으로 검은색 선이 얼굴의 이목구비를 단순하게 표현하고 있으며,...",파블로 피카소는 스페인 말라가에서 태어나 바르셀로나와 마드리드 미술학교를 거쳐 19...
